In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import dspy
import logging
import os
import signal
from typing import List, Union

# Clean up any existing processes on port 5000
os.system('lsof -ti :5000 | xargs kill -9 2>/dev/null')

# Configure Ollama with proper error handling
try:
    dspy.configure(
        lm=dspy.OllamaLocal(
            model="llama3",  # Verified correct model name
            timeout_s=15,
            max_tokens=200,
            temperature=0.5
        )
    )
    logging.info("Ollama configured successfully")
except Exception as e:
    logging.error(f"Failed to configure Ollama: {str(e)}")
    raise SystemExit(1)

app = Flask(__name__)
CORS(app)
logging.basicConfig(level=logging.INFO)

class ColBERTv2:
    def __init__(self, csv_path: str):
        try:
            self.data = pd.read_csv(csv_path)
            if "Ideas" not in self.data.columns:
                raise ValueError("CSV must contain 'Ideas' column")
            self.data["Ideas_lower"] = self.data["Ideas"].str.lower()
            logging.info(f"Loaded {len(self.data)} ideas from CSV")
        except Exception as e:
            logging.error(f"CSV initialization failed: {str(e)}")
            raise

    def __call__(self, query: str, k: int = 3, simplify: bool = True) -> Union[List[str], List[dict]]:
        try:
            query_lower = query.lower()
            matches = self.data[self.data["Ideas_lower"].str.contains(query_lower, na=False)].head(k)
            return matches["Ideas"].tolist() if simplify else [{"text": idea} for idea in matches["Ideas"]]
        except Exception as e:
            logging.error(f"Search failed: {str(e)}")
            return []

try:
    colbertv2_csv = ColBERTv2(csv_path="151_ideas_updated2.csv")
    rag = dspy.ChainOfThought("context, question -> response")
    logging.info("System initialized successfully")
except Exception as e:
    logging.error(f"System initialization failed: {str(e)}")
    raise SystemExit(1)

@app.route('/chat', methods=['POST'])
def chat():
    try:
        data = request.get_json()
        if not data or not data.get('question', '').strip():
            return jsonify({"response": "Invalid request", "context": []}), 400
            
        question = data['question'].strip()
        logging.info(f"Processing question: {question}")
        
        context = colbertv2_csv(question, k=3, simplify=True)
        response = rag(context=context, question=question)
        
        return jsonify({
            "response": str(response.response)[:500],  # Limit response length
            "context": context
        })
        
    except Exception as e:
        logging.error(f"Chat error: {str(e)}")
        return jsonify({"response": "Service unavailable", "context": []}), 500

def shutdown_handler(signum, frame):
    logging.info("Shutting down server...")
    exit(0)

if __name__ == '__main__':
    signal.signal(signal.SIGINT, shutdown_handler)
    signal.signal(signal.SIGTERM, shutdown_handler)
    
    try:
        from waitress import serve
        logging.info("Starting production server on 0.0.0.0:5000")
        serve(app, host="0.0.0.0", port=5000)
    except ImportError:
        logging.warning("Using development server")
        app.run(host='0.0.0.0', port=5000, debug=False)

In [1]:
# app.py
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import logging
import os

# Basic logging configuration
logging.basicConfig(level=logging.INFO)
app = Flask(__name__)
CORS(app)

class SimpleSearchEngine:
    """Simple search engine that loads ideas from CSV and performs basic text matching"""
    def __init__(self, csv_path: str):
        # Load CSV data - ensure your CSV has an 'Ideas' column
        self.data = pd.read_csv(csv_path)
        self.data["Ideas_lower"] = self.data["Ideas"].str.lower()
        logging.info(f"Loaded {len(self.data)} ideas from CSV")

    def search(self, query: str, k: int = 3):
        """Search for ideas containing the query string"""
        query_lower = query.lower()
        matches = self.data[self.data["Ideas_lower"].str.contains(query_lower, na=False)].head(k)
        return matches["Ideas"].tolist()

# Initialize search engine
try:
    search_engine = SimpleSearchEngine("your_ideas.csv")  # Replace with your CSV path
except Exception as e:
    logging.error(f"Failed to initialize search engine: {str(e)}")
    raise SystemExit(1)

@app.route('/chat', methods=['POST'])
def chat():
    try:
        data = request.get_json()
        if not data or 'question' not in data:
            return jsonify({"error": "Invalid request"}), 400

        question = data['question'].strip()
        context = search_engine.search(question)
        
        # In a production environment, you'd want to implement proper AI response generation
        # For now, we'll just return a simple response
        response = f"Based on your question: '{question}', here are some relevant ideas."
        
        return jsonify({
            "response": response,
            "context": context
        })
    except Exception as e:
        logging.error(f"Error processing request: {str(e)}")
        return jsonify({"error": "Internal server error"}), 500

# Deployment Instructions:
"""
DEPLOYMENT OPTIONS:

1. LOCAL DEVELOPMENT:
   - Install requirements: pip install flask flask-cors pandas waitress
   - Run: python app.py
   - Access at: http://localhost:5000

2. RAILWAY DEPLOYMENT (Free Tier):
   - Sign up at railway.app
   - Connect your GitHub repo
   - Add environment variables in Railway dashboard
   - Railway will auto-deploy

3. RENDER DEPLOYMENT (Free Tier):
   - Sign up at render.com
   - Create new Web Service
   - Connect your GitHub repo
   - Add build command: pip install -r requirements.txt
   - Add start command: gunicorn app:app
   - Render will auto-deploy

4. DIGITAL OCEAN APP PLATFORM ($5/month):
   - More reliable than free tiers
   - Sign up at digitalocean.com
   - Create new App
   - Connect your GitHub repo
   - Choose Basic Plan ($5/month)
   - Add environment variables
   - Auto-deploys on push

Requirements for deployment:
- requirements.txt should contain:
  flask==2.0.1
  flask-cors==3.0.10
  pandas==1.3.3
  waitress==2.0.0
  gunicorn==20.1.0
"""

if __name__ == '__main__':
    # Use Waitress for production-grade serving
    from waitress import serve
    serve(app, host="0.0.0.0", port=int(os.environ.get("PORT", 5000)))

ERROR:root:Failed to initialize search engine: [Errno 2] No such file or directory: 'your_ideas.csv'


SystemExit: 1

/opt/homebrew/Caskroom/miniforge/base/envs/nunu24/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
